In [1]:
from imp import reload
# reload(dst)
import torch
from torch.utils.data import DataLoader
import cs231n.datasets.local_CIFAR10 as dst
import torchvision.transforms as T
import cs231n.datasets.local_CIFAR10
from torch.utils.data import sampler

This application is used to convert notebook files (*.ipynb) to various other
formats.


Options

-------



Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.


--debug

    set log level to logging.DEBUG (maximize logging output)

--generate-config

    generate default config file

-y

    Answer yes to any questions instead of prompting.

--execute

    Execute the notebook prior to export.

--allow-errors

    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.

--stdin

    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'

--stdout

    Write notebook output to stdout instead of files.

--inplace

    Run nbconvert in 

[NbConvertApp] WARNING | pattern 'local_CIFAR10.ipynb' matched no files


In [2]:
import sys
sys.path.append("E:/JupyterEnviroment/Learning/MachineLearning/Network/cs231n")

In [10]:
NUM_TRAIN = 49000
transform = T.Compose([T.ToTensor(),T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
train_data = dst.local_CIFAR10(root="C:/Users/wtser/Desktop/learnData/data/",transform=transform,train=True)
load_train = DataLoader(dataset=train_data,batch_size=64)
test_data = dst.local_CIFAR10(root="C:/Users/wtser/Desktop/learnData/data/",transform=transform,train=False)
load_test = DataLoader(dataset=test_data,batch_size=64,shuffle=True)

In [7]:
class SimpleCNN(torch.nn.Module):
    def __init__(self,num_classes):
        super(SimpleCNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(8*8*32,num_classes)
    
    def forward(self,x):
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out2 = out2.reshape(out2.shape[0],-1)
        out3 = self.fc(out2)
        return out3

In [8]:
device = torch.device('cuda')
model = SimpleCNN(10).to(device)

In [13]:
num_epochs = 10
num_classes = 10
batch_size = 100
learning_rate = 0.001
total_step = len(load_train)
model = SimpleCNN(10).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimer = torch.optim.Adam(model.parameters(),lr=learning_rate)
for epoch in range(num_epochs):
    for i,(x,y) in enumerate(load_train):
        x = x.to(device)
        y = y.to(device)
        out = model(x)
        loss = criterion(out,y)
    
        optimer.zero_grad()
        loss.backward()
        optimer.step()
        if i%100 == 0:
            print('Epoch[{}/{}],Step[{}/{}],loss{:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

#test Model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i,(x,y) in enumerate(load_test):
        x = x.to(device)
        y = y.to(device)
        out = model(x)
        _,pre = torch.max(out,1)
        total += y.shape[0]
        correct += (pre==y).sum().item()
    print('Test accuracy of model on 10000 of test images:{}%'.format(100*correct/total ))

Epoch[1/10],Step[1/782],loss2.3544
Epoch[1/10],Step[101/782],loss1.5184
Epoch[1/10],Step[201/782],loss1.2354
Epoch[1/10],Step[301/782],loss1.1999
Epoch[1/10],Step[401/782],loss1.1962
Epoch[1/10],Step[501/782],loss1.3901
Epoch[1/10],Step[601/782],loss1.2288
Epoch[1/10],Step[701/782],loss1.0825
Epoch[2/10],Step[1/782],loss1.0440
Epoch[2/10],Step[101/782],loss0.9515
Epoch[2/10],Step[201/782],loss0.8346
Epoch[2/10],Step[301/782],loss0.9639
Epoch[2/10],Step[401/782],loss0.9484
Epoch[2/10],Step[501/782],loss1.2476
Epoch[2/10],Step[601/782],loss1.0005
Epoch[2/10],Step[701/782],loss0.9676
Epoch[3/10],Step[1/782],loss0.9262
Epoch[3/10],Step[101/782],loss0.7933
Epoch[3/10],Step[201/782],loss0.7798
Epoch[3/10],Step[301/782],loss0.8343
Epoch[3/10],Step[401/782],loss0.8316
Epoch[3/10],Step[501/782],loss1.1181
Epoch[3/10],Step[601/782],loss0.9007
Epoch[3/10],Step[701/782],loss0.8816
Epoch[4/10],Step[1/782],loss0.8598
Epoch[4/10],Step[101/782],loss0.7109
Epoch[4/10],Step[201/782],loss0.7514
Epoch[4/1